In [60]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import math

In [61]:
dt_train=r'D:/AI & Data Science/Python/Project 2/carvan_train.csv'
dt_test=r'D:/AI & Data Science/Python/Project 2/carvan_test.csv'
cd_train=pd.read_csv(dt_train)
cd_test=pd.read_csv(dt_test)

In [62]:
cd_train.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86
0,33,1,3,2,8,0,5,1,3,7,...,0,0,0,1,0,0,0,0,0,0
1,37,1,2,2,8,1,4,1,4,6,...,0,0,0,1,0,0,0,0,0,0
2,37,1,2,2,8,0,4,2,4,3,...,0,0,0,1,0,0,0,0,0,0
3,9,1,3,3,3,2,3,2,4,5,...,0,0,0,1,0,0,0,0,0,0
4,40,1,4,2,10,1,4,1,4,7,...,0,0,0,1,0,0,0,0,0,0


In [63]:
cd_test

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85
0,33,1,4,2,8,0,6,0,3,5,...,0,0,0,0,1,0,0,0,0,0
1,6,1,3,2,2,0,5,0,4,5,...,2,0,0,0,1,0,0,0,0,0
2,39,1,3,3,9,1,4,2,3,5,...,1,0,0,0,1,0,0,0,0,0
3,9,1,2,3,3,2,3,2,4,5,...,0,0,0,0,1,0,0,0,0,0
4,31,1,2,4,7,0,2,0,7,9,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,33,1,2,4,8,0,7,2,0,5,...,0,0,0,0,1,0,0,0,0,0
3996,24,1,2,3,5,1,5,1,3,4,...,1,0,0,0,1,0,0,0,0,0
3997,36,1,2,3,8,1,5,1,3,7,...,0,0,0,0,1,0,0,0,1,0
3998,33,1,3,3,8,1,4,2,3,7,...,0,0,0,0,0,0,0,0,0,0


In [64]:
from sklearn.model_selection import train_test_split,KFold
from sklearn.linear_model import LogisticRegression
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
import numpy as np
from sklearn.model_selection import KFold
%matplotlib inline
from sklearn.metrics import roc_auc_score

In [65]:
bd_train, bd_test = train_test_split(cd_train, test_size = 0.2,random_state=2)

In [66]:
x_train=bd_train.drop(["V86"],1)
y_train=bd_train["V86"]
x_test=bd_test.drop(["V86"],1)
y_test=bd_test["V86"]

In [67]:
logr=LogisticRegression(penalty="l1",class_weight="balanced",random_state=2)

In [68]:
logr.fit(x_train,y_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l1',
                   random_state=2, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [69]:
logr.predict_proba(x_test)

array([[0.46269231, 0.53730769],
       [0.55846019, 0.44153981],
       [0.22977718, 0.77022282],
       ...,
       [0.2325136 , 0.7674864 ],
       [0.82976088, 0.17023912],
       [0.74741265, 0.25258735]])

In [73]:
logr.predict(x_test)

array([1, 0, 1, ..., 1, 0, 0], dtype=int64)

In [70]:
roc_auc_score(y_test,logr.predict(x_test))

0.6759633027522937

In [71]:
prob_score=pd.Series(list(zip(*logr.predict_proba(x_train)))[1])

In [72]:
prob_score

0       0.391726
1       0.055335
2       0.444702
3       0.187836
4       0.297118
          ...   
4652    0.060818
4653    0.643363
4654    0.550209
4655    0.136016
4656    0.305928
Length: 4657, dtype: float64

In [74]:
cutoffs=np.linspace(0,1,100)

In [76]:
KS_cut=[]
for cutoff in cutoffs:
    predicted=pd.Series([0]*len(y_train))
    predicted[prob_score>cutoff]=1
    df=pd.DataFrame(list(zip(y_train,predicted)),
                    columns=["real","predicted"])
    TP=len(df[(df["real"]==1) &(df["predicted"]==1) ])
    FP=len(df[(df["real"]==0) &(df["predicted"]==1) ])
    TN=len(df[(df["real"]==0) &(df["predicted"]==0) ])
    FN=len(df[(df["real"]==1) &(df["predicted"]==0) ])
    P=TP+FN
    N=TN+FP
    KS=(TP/P)-(FP/N)
    KS_cut.append(KS)

cutoff_data=pd.DataFrame(list(zip(cutoffs,KS_cut)),columns=["cutoff","KS"])

KS_cutoff=cutoff_data[cutoff_data["KS"]==cutoff_data["KS"].max()]["cutoff"]

In [77]:
KS_cutoff

56    0.565657
Name: cutoff, dtype: float64

In [81]:
# Performance on test data
prob_score_test=pd.Series(list(zip(*logr.predict_proba(x_test)))[1])

predicted_test=pd.Series([0]*len(y_test))
predicted_test[prob_score_test>float(KS_cutoff)]=1

df_test=pd.DataFrame(list(zip(y_test,predicted_test)),
                     columns=["real","predicted"])

k=pd.crosstab(df_test['real'],df_test["predicted"])
print('confusion matrix :\n \n ',k)
TN=k.iloc[0,0]
TP=k.iloc[1,1]
FP=k.iloc[0,1]
FN=k.iloc[1,0]
P=TP+FN
N=TN+FP

confusion matrix :
 
  predicted    0    1
real               
0          853  237
1           36   39


In [82]:
# Accuracy of test
(TP+TN)/(P+N)

0.7656652360515022

In [84]:
# Sensitivity on test
TP/P

0.52

In [83]:
#Specificity on test
TN/N

0.7825688073394496

In [85]:
cutoffs=np.linspace(0.010,0.99,100)
def Fbeta_perf(beta,cutoffs,y_train,prob_score):
    FB_cut=[]
    for cutoff in cutoffs:
        predicted=pd.Series([0]*len(y_train))
        predicted[prob_score>cutoff]=1
        df=pd.DataFrame(list(zip(y_train,predicted)),columns=["real","predicted"])

        TP=len(df[(df["real"]==1) &(df["predicted"]==1) ])
        FP=len(df[(df["real"]==0) &(df["predicted"]==1) ])
        FN=len(df[(df["real"]==1) &(df["predicted"]==0) ])
        P=TP+FN
        
        
        Precision=TP/(TP+FP)
        Recall=TP/P
        FB=(1+beta**2)*Precision*Recall/((beta**2)*Precision+Recall)
        FB_cut.append(FB)

    cutoff_data=pd.DataFrame(list(zip(cutoffs,FB_cut)),columns=["cutoff","FB"])

    FB_cutoff=cutoff_data[cutoff_data["FB"]==cutoff_data["FB"].max()]["cutoff"]

    prob_score_test=pd.Series(list(zip(*logr.predict_proba(x_test)))[1])

    predicted_test=pd.Series([0]*len(y_test))
    predicted_test[prob_score_test>float(FB_cutoff)]=1

    df_test=pd.DataFrame(list(zip(y_test,predicted_test)),columns=["real","predicted"])

    k=pd.crosstab(df_test['real'],df_test["predicted"])
#     print('confusion matrix :\n \n ',k)
    TN=k.iloc[0,0]
    TP=k.iloc[1,1]
    FP=k.iloc[0,1]
    FN=k.iloc[1,0]
    P=TP+FN
    N=TN+FP
    print('For beta :',beta)
    print('Accuracy is :',(TP+TN)/(P+N))
    print('Sensitivity is :',(TP/P))
    print('Specificity is :',(TN/N))
    print('\n \n \n')

In [86]:
Fbeta_perf(0.5,cutoffs,y_train,prob_score) #false +ve
Fbeta_perf(1,cutoffs,y_train,prob_score) #false -ve 
Fbeta_perf(2,cutoffs,y_train,prob_score) #both

For beta : 0.5
Accuracy is : 0.9150214592274678
Sensitivity is : 0.22666666666666666
Specificity is : 0.9623853211009175

 
 

For beta : 1
Accuracy is : 0.8437768240343347
Sensitivity is : 0.4266666666666667
Specificity is : 0.8724770642201835

 
 

For beta : 2
Accuracy is : 0.7733905579399142
Sensitivity is : 0.49333333333333335
Specificity is : 0.7926605504587156

 
 



In [87]:
pre=TP/(TP+FP)
pre

0.14130434782608695

In [88]:
rec=TP/P
rec

0.52

In [89]:
((1+2)*pre*rec)/(2*pre+rec)

0.2746478873239437

In [90]:
final=logr.predict_proba(cd_test)

In [102]:
final_1=logr.predict(cd_test)

In [91]:
final

array([[0.81128132, 0.18871868],
       [0.15794017, 0.84205983],
       [0.50128345, 0.49871655],
       ...,
       [0.28811069, 0.71188931],
       [0.50796149, 0.49203851],
       [0.38572093, 0.61427907]])

In [103]:
final_1

array([0, 1, 0, ..., 1, 0, 1], dtype=int64)

In [95]:
final_score=pd.Series(list(zip(*logr.predict_proba(cd_test)))[1])

In [96]:
final_score

0       0.188719
1       0.842060
2       0.498717
3       0.585434
4       0.160550
          ...   
3995    0.551278
3996    0.393557
3997    0.711889
3998    0.492039
3999    0.614279
Length: 4000, dtype: float64

In [97]:
final_test=pd.Series([0]*len(cd_test))

In [98]:
final_test

0       0
1       0
2       0
3       0
4       0
       ..
3995    0
3996    0
3997    0
3998    0
3999    0
Length: 4000, dtype: int64

In [99]:
final_test[final_score>float(KS_cutoff)]=1

In [100]:
final_test

0       0
1       1
2       0
3       1
4       0
       ..
3995    0
3996    0
3997    1
3998    0
3999    1
Length: 4000, dtype: int64

In [104]:
roc_auc_score(final_1,final_test)

0.8914285714285715

In [105]:
submission=pd.DataFrame(list(final_test),columns=['V86'])

In [106]:
submission.to_csv('Abhilash_Singh_P2_part2.csv',columns=['V86'],index=False)